In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import ta 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Input, GRU, Dense,LSTM
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Making a Function to Forecast next 24 timesteps

#

In [11]:
df=pd.read_csv("/Users/rohan/Documents/GitHub/RT-Sem-2/Dataset/1H(2019-2024).csv")
df.drop(["Volume"],axis=1,inplace=True)

df.rename(columns={'Gmt time' : 'timestamp', 'Open' : 'open', 
                    'High' : 'high', 'Low' : 'low', 'Close' : 'close'}, inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)
df.set_index('timestamp', inplace=True)
df = df.astype(float)

df['hour'] = df.index.hour
df['day']  = df.index.weekday # Day of the week
df['week'] = df.index.to_series().dt.isocalendar().week
df['week'] = df.index.isocalendar().week
df['momentum']  = (df['open'] - df['close'])
df['avg_price'] = (df['low'] + df['high'])/2
df['range']     = df['high'] - df['low']
df['ohlc_price'] = (df['low'] + df['high'] + df['open'] + df['close'])/4
df['rsi'] = ta.momentum.RSIIndicator(close = df['close'], window = 14).rsi()
df['macd'] = ta.trend.MACD(close = df['close']).macd()

df.dropna(inplace=True)


def create_dataset(dataset, look_back=30):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

    
open_index = df.columns.tolist().index('open')
close_index = df.columns.tolist().index('close')
high_index = df.columns.tolist().index('high')
low_index = df.columns.tolist().index('low')
dataset = df.values.astype('float32')

/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2303498356.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


# Making a General Function for predicting High, Low, Close 

In [12]:

def pred(index,weights):
    
    df=pd.read_csv("/Users/rohan/Documents/GitHub/RT-Sem-2/Dataset/1H(2019-2024).csv")
    df.drop(["Volume"],axis=1,inplace=True)

    df.rename(columns={'Gmt time' : 'timestamp', 'Open' : 'open', 
                    'High' : 'high', 'Low' : 'low', 'Close' : 'close'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)

    df['hour'] = df.index.hour
    df['day']  = df.index.weekday # Day of the week
    df['week'] = df.index.to_series().dt.isocalendar().week
    df['week'] = df.index.isocalendar().week
    df['momentum']  = (df['open'] - df['close'])
    df['avg_price'] = (df['low'] + df['high'])/2
    df['range']     = df['high'] - df['low']
    df['ohlc_price'] = (df['low'] + df['high'] + df['open'] + df['close'])/4
    df['rsi'] = ta.momentum.RSIIndicator(close = df['close'], window = 14).rsi()
    df['macd'] = ta.trend.MACD(close = df['close']).macd()

    df.dropna(inplace=True)


    def create_dataset(dataset, look_back=30):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    
    open_index = df.columns.tolist().index('open')
    close_index = df.columns.tolist().index('close')
    high_index = df.columns.tolist().index('high')
    low_index = df.columns.tolist().index('low')
    dataset = df.values.astype('float32')

    X, y = create_dataset(dataset, look_back=30)
    y = y[:,index]

    train_size = int(len(X) * 0.80)
    print(len(X))
    trainX = X[:train_size]
    trainY = y[:train_size]
    testX = X[train_size:]
    testY = y[train_size:]

    scaler_X=MinMaxScaler(feature_range=(0,1))
    scaler_X=scaler_X.fit(trainX.reshape(-1, trainX.shape[-1]))
    trainX_scaled = scaler_X.transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX_scaled = scaler_X.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    trainX=trainX_scaled
    testX=testX_scaled

    scaler_y=MinMaxScaler(feature_range=(0,1))
    scaler_y=scaler_y.fit(trainY.reshape(-1, 1))
    trainY_scaled = scaler_y.transform(trainY.reshape(-1, 1))
    testY_scaled = scaler_y.transform(testY.reshape(-1, 1))
    trainY=trainY_scaled
    testY=testY_scaled

    model = Sequential()
    model.add(GRU(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dropout(0.3))
    model.add(Dense(64 ,  activation = 'relu'))
    model.add(Dense(32,  activation = 'relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    # print(model.summary())

    import os

    # Change this by your folder directory
    new_directory = "/Users/rohan/Documents/GitHub/RT-Sem-2/Python Codes"

    # Change the current working directory
    os.chdir(new_directory)

    import os
    checkpoint_path=f"{weights}.weights.h5"
    checkpoint_dir=os.path.dirname(checkpoint_path)

    os.getcwdb()

    model.load_weights(checkpoint_path)

    pred = model.predict(testX)
    
    pred_test_inv=scaler_y.inverse_transform(pred)
    pred_test_inv=np.round(pred_test_inv,5)

    return pred_test_inv[-1]

## We have a lookback of 30 so we ll now Forecast 31st value

In [13]:
pred_close=pred(close_index,"open15")
pred_high=pred(high_index,"weights15")
pred_low=pred(low_index,"low15")
df=pd.read_csv("/Users/rohan/Documents/GitHub/RT-Sem-2/Dataset/1H(2019-2024).csv")
pred_open=df.iloc[-1]["Close"] # Using the previous close as open of current timestep


# convert this to number pred_close,pred_high,pred_low
pred_close=np.round(float(pred_close),5)
pred_high=np.round(float(pred_high),5)
pred_low=np.round(float(pred_low),5)

print(pred_open,pred_high,pred_low,pred_close)

/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2190187025.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


32630


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/homebrew/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2190187025.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


...

/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2190187025.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


32630


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/homebrew/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step
1.09373 1.09224 1.09084 1.09155


/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2465169114.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_close=np.round(float(pred_close),5)
/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2465169114.py:10: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_high=np.round(float(pred_high),5)
/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2465169114.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pre

### Our Forecasted values are:

In [14]:
print(pred_open,pred_high,pred_low,pred_close)

1.09373 1.09224 1.09084 1.09155


In [15]:
df1=pd.read_csv("/Users/rohan/Documents/GitHub/RT-Sem-2/Dataset/1H(2019-2024).csv")
df1.drop(["Volume"],axis=1,inplace=True)
l=["2024-03-21 00:00:00.000",pred_open,pred_high,pred_low,pred_close];l
df1.loc[len(df1)]=l
df1


,Gmt time,Open,High,Low,Close
0,01.01.2019 00:00:00.000,1.14627,1.14627,1.14627,1.14627
1,01.01.2019 01:00:00.000,1.14627,1.14627,1.14627,1.14627
2,01.01.2019 02:00:00.000,1.14627,1.14627,1.14627,1.14627
3,01.01.2019 03:00:00.000,1.14627,1.14627,1.14627,1.14627
4,01.01.2019 04:00:00.000,1.14627,1.14627,1.14627,1.14627
...,...,...,...,...,...
32682,20.03.2024 20:00:00.000,1.09209,1.09228,1.09187,1.09214
32683,20.03.2024 21:00:00.000,1.09214,1.09229,1.09179,1.09206
32684,20.03.2024 22:00:00.000,1.09206,1.09323,1.09203,1.09321
32685,20.03.2024 23:00:00.000,1.09320,1.09374,1.09293,1.09373


In [16]:
ts=["2024-03-21 01:00:00.000",'2024-03-21 02:00:00.000','2024-03-21 03:00:00.000','2024-03-21 04:00:00.000','2024-03-21 05:00:00.000',"2024-03-21 06:00:00.000","2024-03-21 07:00:00.000","2024-03-21 08:00:00.000","2024-03-21 09:00:00.000","2024-03-21 10:00:00.000",
    "2024-03-21 11:00:00.000","2024-03-21 12:00:00.000","2024-03-21 13:00:00.000","2024-03-21 14:00:00.000","2024-03-21 15:00:00.000","2024-03-21 16:00:00.000","2024-03-21 17:00:00.000","2024-03-21 18:00:00.000","2024-03-21 19:00:00.000","2024-03-21 20:00:00.000","2024-03-21 21:00:00.000","2024-03-21 22:00:00.000","2024-03-21 23:00:00.000","2024-03-22 00:00:00.000","2024-03-22 01:00:00:000"]


# Again Generalising the model to Forecast 

In [17]:
def pred1(df,index,weights):
    
    df=df1.copy()
    

    df.rename(columns={'Gmt time' : 'timestamp', 'Open' : 'open', 
                    'High' : 'high', 'Low' : 'low', 'Close' : 'close'}, inplace=True)
    df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)
    df.set_index('timestamp', inplace=True)
    df = df.astype(float)

    df['hour'] = df.index.hour
    df['day']  = df.index.weekday # Day of the week
    df['week'] = df.index.to_series().dt.isocalendar().week
    df['week'] = df.index.isocalendar().week
    df['momentum']  = (df['open'] - df['close'])
    df['avg_price'] = (df['low'] + df['high'])/2
    df['range']     = df['high'] - df['low']
    df['ohlc_price'] = (df['low'] + df['high'] + df['open'] + df['close'])/4
    df['rsi'] = ta.momentum.RSIIndicator(close = df['close'], window = 14).rsi()
    df['macd'] = ta.trend.MACD(close = df['close']).macd()

    df.dropna(inplace=True)


    def create_dataset(dataset, look_back=30):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back)]
            dataX.append(a)
            dataY.append(dataset[i + look_back])
        return np.array(dataX), np.array(dataY)

    
    open_index = df.columns.tolist().index('open')
    close_index = df.columns.tolist().index('close')
    high_index = df.columns.tolist().index('high')
    low_index = df.columns.tolist().index('low')
    dataset = df.values.astype('float32')

    X, y = create_dataset(dataset, look_back=30)
    y = y[:,index]

    train_size = int(len(X) * 0.80)
    print(len(X))
    trainX = X[:train_size]
    trainY = y[:train_size]
    testX = X[train_size:]
    testY = y[train_size:]

    scaler_X=MinMaxScaler(feature_range=(0,1))
    scaler_X=scaler_X.fit(trainX.reshape(-1, trainX.shape[-1]))
    trainX_scaled = scaler_X.transform(trainX.reshape(-1, trainX.shape[-1])).reshape(trainX.shape)
    testX_scaled = scaler_X.transform(testX.reshape(-1, testX.shape[-1])).reshape(testX.shape)
    trainX=trainX_scaled
    testX=testX_scaled

    scaler_y=MinMaxScaler(feature_range=(0,1))
    scaler_y=scaler_y.fit(trainY.reshape(-1, 1))
    trainY_scaled = scaler_y.transform(trainY.reshape(-1, 1))
    testY_scaled = scaler_y.transform(testY.reshape(-1, 1))
    trainY=trainY_scaled
    testY=testY_scaled

    model = Sequential()
    model.add(GRU(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dropout(0.3))
    model.add(Dense(64 ,  activation = 'relu'))
    model.add(Dense(32,  activation = 'relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    # print(model.summary())

    import os

    # Change this by your folder directory
    new_directory = "/Users/rohan/Documents/GitHub/RT-Sem-2/Python Codes"

    # Change the current working directory
    os.chdir(new_directory)

    import os
    checkpoint_path=f"{weights}.weights.h5"
    checkpoint_dir=os.path.dirname(checkpoint_path)

    os.getcwdb()

    model.load_weights(checkpoint_path)

    pred = model.predict(testX)
    
    pred_test_inv=scaler_y.inverse_transform(pred)
    pred_test_inv=np.round(pred_test_inv,5)

    return pred_test_inv[-1]

# Now Forecasting next 24 timesteps

In [18]:

for i in range(1,len(ts)):
    pred_close=pred1(df1,close_index,"open15")
    pred_high=pred1(df1,high_index,"weights15")
    pred_low=pred1(df1,low_index,"low15")
    pred_open=df1.iloc[-1]["Close"]

    pred_close=np.round(float(pred_close),5)
    pred_high=np.round(float(pred_high),5)
    pred_low=np.round(float(pred_low),5)
    df1.loc[len(df1)]=[ts[i],pred_open,pred_high,pred_low,pred_close]
    
display(df1.tail(24))


/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/223178986.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


32631
  1/204 ━━━━━━━━━━━━━━━━━━━━ 27s 136ms/step

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/homebrew/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/223178986.py:8: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['timestamp'] = pd.to_datetime(df['timestamp'],format="mixed",infer_datetime_format=True)


...

32654
  1/205 ━━━━━━━━━━━━━━━━━━━━ 27s 136ms/step

/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/opt/homebrew/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


205/205 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2146088326.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_close=np.round(float(pred_close),5)
/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2146088326.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_high=np.round(float(pred_high),5)
/var/folders/l_/_lklt0y16gxfy4_c8rk_0chw0000gn/T/ipykernel_23481/2146088326.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred_

,Gmt time,Open,High,Low,Close
32687,2024-03-21 02:00:00.000,1.09155,1.09327,1.09203,1.09257
32688,2024-03-21 03:00:00.000,1.09257,1.09368,1.09225,1.09291
32689,2024-03-21 04:00:00.000,1.09291,1.09167,1.08968,1.09119
32690,2024-03-21 05:00:00.000,1.09119,1.09224,1.09090,1.09188
32691,2024-03-21 06:00:00.000,1.09188,1.09263,1.09142,1.09226
32692,2024-03-21 07:00:00.000,1.09226,1.09108,1.08912,1.09079
32693,2024-03-21 08:00:00.000,1.09079,1.09154,1.09014,1.09114
32694,2024-03-21 09:00:00.000,1.09114,1.09196,1.09075,1.09158
32695,2024-03-21 10:00:00.000,1.09158,1.09067,1.08891,1.09036
32696,2024-03-21 11:00:00.000,1.09036,1.09092,1.08948,1.09051
